In [47]:
from __future__ import division

import numpy as np
import random



class NeuralNetwork(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes # [2,3,1]
        'biases. loop creates a list [] of arrays of size [y,1] for every given layer size, except the input one [1:], which is the 0th position'
        self.biases = [np.random.randn(x,1) for x in sizes[1:]]
        'zip associates the input layer for x with the second for y [1:] until the previous of the last for x [:-1] with the last one for y'
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1], sizes[1:])]



    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, lr, test_data=None):
       
        test_data = list(test_data)
        if test_data: n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data) #shuffing occurs at each epoch
            'mini_batches is a list of equally divided training_data from 0 to n using mini_bacth_size for every step'
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, lr)

            if test_data:
                print ("Epoch {0}: {1}/{2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, lr):

        'variables to store the gradients'
        gradient_b = [np.zeros(b.shape) for b in self.biases]
        gradient_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_gradient_b, delta_gradient_w = self.backprop(x,y)
            'storing computed gradients'
            'gb+dgb every time because this calculates the summatory for all examples x'
            gradient_b = [gb+dgb for gb, dgb in zip(gradient_b, delta_gradient_b)]
            gradient_w = [gw+dgw for gw, dgw in zip(gradient_w, delta_gradient_w)]

        'updating values with gradient descent'
        'just *gb because it holds the summatory value already done previously'
        self.biases = [b-(lr/len(mini_batch))*gb for b, gb in zip(self.biases, gradient_b)]
        self.weights = [w-(lr/len(mini_batch))*gw for w, gw in zip(self.weights, gradient_w)]

    def backprop(self, x, y):

            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.weights]
            # feedforward
            activation = x
            activations = [x] # list to store all the activations, layer by layer
            zs = [] # list to store all the z vectors, layer by layer
            for b, w in zip(self.biases, self.weights):
                z = np.dot(w, activation)+b
                zs.append(z)
                activation = sigmoid(z)
                activations.append(activation)
            # backward pass
            delta = self.cost_derivative(activations[-1], y) * \
                sigmoid_prime(zs[-1])
            nabla_b[-1] = delta
            nabla_w[-1] = np.dot(delta, activations[-2].transpose())

            for l in range(2, self.num_layers):
                z = zs[-l]
                sp = sigmoid_prime(z)
                delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
                nabla_b[-l] = delta
                nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        'argmax returns index of max activation neuron on ouput layer, in this case it coincides with the position, so argmax can be used for junt this case'
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x==y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

def sigmoid(z):
        return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
        'derivate of sigmoid function'
        return sigmoid(z)*(1-sigmoid(z))

In [48]:
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f,encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [49]:
training_data, validation_data, test_data = load_data_wrapper()

In [50]:
net = NeuralNetwork([784, 100, 10])

In [51]:
net.SGD(training_data, 40, 10, 3.0, test_data=test_data)

Epoch 0: 6368/10000
Epoch 1: 7371/10000
Epoch 2: 7406/10000
Epoch 3: 7463/10000
Epoch 4: 7499/10000
Epoch 5: 7504/10000
Epoch 6: 7531/10000
Epoch 7: 7517/10000
Epoch 8: 7566/10000
Epoch 9: 7562/10000
Epoch 10: 7560/10000
Epoch 11: 7600/10000
Epoch 12: 7566/10000
Epoch 13: 7614/10000
Epoch 14: 8704/10000
Epoch 15: 8728/10000
Epoch 16: 9521/10000
Epoch 17: 9537/10000
Epoch 18: 9583/10000
Epoch 19: 9579/10000
Epoch 20: 9616/10000
Epoch 21: 9601/10000
Epoch 22: 9604/10000
Epoch 23: 9606/10000
Epoch 24: 9610/10000
Epoch 25: 9608/10000
Epoch 26: 9615/10000
Epoch 27: 9616/10000
Epoch 28: 9616/10000
Epoch 29: 9626/10000
Epoch 30: 9634/10000
Epoch 31: 9630/10000
Epoch 32: 9619/10000
Epoch 33: 9638/10000
Epoch 34: 9626/10000
Epoch 35: 9627/10000
Epoch 36: 9634/10000
Epoch 37: 9640/10000
Epoch 38: 9637/10000
Epoch 39: 9652/10000
